In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from collections import Counter

import json

In [ ]:
df = pd.read_excel("/kaggle/input/cdc-2021-text-sequence-dataset/Text_Sequence_Dataset.xlsx")

In [ ]:
df.info()

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(df['Sentence'], df['Classification'], test_size=0.20, random_state=42)

label_encoder = LabelEncoder().fit(train_labels)
train_labels = label_encoder.transform(train_labels)
test_labels = label_encoder.transform(test_labels)

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_train_encodings = bert_tokenizer(train_texts.tolist(), truncation=True, padding=True, return_tensors='pt')
bert_test_encodings = bert_tokenizer(test_texts.tolist(), truncation=True, padding=True, return_tensors='pt')

bert_train_dataset = TensorDataset(bert_train_encodings['input_ids'], bert_train_encodings['attention_mask'], torch.tensor(train_labels))
bert_test_dataset = TensorDataset(bert_test_encodings['input_ids'], bert_test_encodings['attention_mask'], torch.tensor(test_labels))

bert_train_loader = DataLoader(bert_train_dataset, batch_size=8, shuffle=True)
bert_test_loader = DataLoader(bert_test_dataset, batch_size=8, shuffle=False)

gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
gpt2_train_encodings = gpt2_tokenizer(train_texts.tolist(), truncation=True, padding=True, return_tensors='pt')
gpt2_test_encodings = gpt2_tokenizer(test_texts.tolist(), truncation=True, padding=True, return_tensors='pt')

gpt2_train_dataset = TensorDataset(gpt2_train_encodings['input_ids'], gpt2_train_encodings['attention_mask'], torch.tensor(train_labels))
gpt2_test_dataset = TensorDataset(gpt2_test_encodings['input_ids'], gpt2_test_encodings['attention_mask'], torch.tensor(test_labels))

gpt2_train_loader = DataLoader(gpt2_train_dataset, batch_size=8, shuffle=True)
gpt2_test_loader = DataLoader(gpt2_test_dataset, batch_size=8, shuffle=False)

softmax = torch.nn.Softmax(dim=-1) # a surprise tool that will help us later

In [ ]:

bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

class_weights = torch.tensor([1.0, 1.0], dtype=torch.float)

optimizer = torch.optim.AdamW(bert_model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

bert_model.train()
for epoch in range(4):  # Four epochs
    for batch in bert_train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = bert_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


In [ ]:

bert_model.eval()
correct = 0
total = 0

pred = []
prob = []
true = []

with torch.no_grad():
    for batch in bert_test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = bert_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pred = pred + predicted.tolist()
        prob = prob + softmax(outputs.logits).tolist()
        true = true + labels.tolist()

accuracy = correct / total
print(f'Accuracy on test set: {accuracy}')

with open("bert_results.json","w") as f:
    json.dump({"predicted":pred,"probabilities":prob,"labels":true},f)


In [ ]:

gpt2_model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
gpt2_model.config.pad_token_id = gpt2_model.config.eos_token_id
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpt2_model.to(device)

class_weights = torch.tensor([1.0, 1.0], dtype=torch.float)

optimizer = torch.optim.AdamW(gpt2_model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

gpt2_model.train()
for epoch in range(4):
    for batch in gpt2_train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = gpt2_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        

In [ ]:

gpt2_model.eval()
correct = 0
total = 0

pred = []
prob = []
true = []

with torch.no_grad():
    for batch in gpt2_test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = gpt2_model(input_ids, attention_mask=attention_mask)
        _, predicted = torch.max(outputs.logits, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pred = pred + predicted.tolist()
        prob = prob + softmax(outputs.logits).tolist()
        true = true + labels.tolist()

accuracy = correct / total
print(f'Accuracy on test set: {accuracy}')

with open("gpt2_results.json","w") as f:
    json.dump({"predicted":pred,"probabilities":prob,"labels":true},f)
